In [28]:

import sqlite3
import pandas as pd
from bokeh.plotting import figure, save, output_file, curdoc
from bokeh.models import ColumnDataSource, HoverTool, ColorBar, BasicTicker, PrintfTickFormatter, Div
from bokeh.transform import linear_cmap
from bokeh.layouts import column
from bokeh.palettes import Viridis256
import numpy as np

# Connect to SQLite database
conn = sqlite3.connect('/Users/leomarcrowal/GitHuB_Local/Project-3/static/Resources/new_aqi_ngsDB.sqlite')

# Write an SQL query
query = "SELECT AQI, total, month, state_name FROM combined_df"

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(query, conn)

# Calculate the correlation coefficient between AQI and total
correlation = df['AQI'].corr(df['total'])
print(f"Correlation coefficient between AQI and total: {correlation}")

# Prepare data for Bokeh
source = ColumnDataSource(df)

# Calculate linear regression
slope, intercept = np.polyfit(df['AQI'], df['total'], 1)
regression_line = slope * df['AQI'] + intercept

# Create a new scatter plot
p = figure(title=f'Scatter Plot of AQI vs Total (Correlation: {correlation:.2f})',
           x_axis_label='AQI', y_axis_label='Total', width=700, height=400,
           tools="pan,wheel_zoom,box_zoom,reset,save,hover", active_drag="box_zoom", active_scroll="wheel_zoom")

# Map the AQI values to colors using a new linear color mapper
mapper = linear_cmap(field_name='AQI', palette=Viridis256, low=df['AQI'].min(), high=df['AQI'].max())

# Add scatter glyphs to the plot using scatter() instead of circle()
p.scatter(x='AQI', y='total', size=8, color=mapper, source=source, line_alpha=0.6, fill_alpha=0.6, 
          selection_color="firebrick", nonselection_fill_alpha=0.2, nonselection_line_alpha=0.2)

# Add the linear regression line
p.line(df['AQI'], regression_line, line_width=2, color='red', legend_label='Linear Regression')

# Add a color bar to show the AQI color scale
color_bar = ColorBar(color_mapper=mapper['transform'], width=8, location=(0,0),
                     ticker=BasicTicker(desired_num_ticks=10),
                     formatter=PrintfTickFormatter(format="%d"))

p.add_layout(color_bar, 'right')

# Add tooltips to show information on hover
p.hover.tooltips = [
    ("State", "@state_name"),
    ("Month", "@month"),
    ("AQI", "@AQI"),
    ("Total", "@total"),
]

# Create a Div widget to display the comparison text
div_text = Div(
    text="""
    <div style="text-align: center; background-color:lightgreen; padding: 10px;">
        <h2>Low Correlation Between Air Quality Index and Natural Gas Consumption</h2>
    </div>
        """,
    width=2000,
    height=50
)

# Combine plot and div into a layout
layout = column(div_text, p)

# Ensure Bokeh plots are saved to an HTML file
output_file("/Users/leomarcrowal/GitHuB_Local/Project-3/static/pages/correlation_of_AQI_vs_TotalNGas.html")

# Use a new document to save the plot
curdoc().add_root(layout)
save(layout)


Correlation coefficient between AQI and total: 0.09918364845115557


'/Users/leomarcrowal/GitHuB_Local/Project-3/static/pages/correlation_of_AQI_vs_TotalNGas.html'